In [1]:
import qforte as qf

$$
\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}
$$

# QForte's state-vector simulator

State vector simulators are among some of the most common quantum computer simulators employed today. 
While many such simulators rely (mostly) on sparse tensor operations, QForte takes an approach that closer resembles modern FCI implementations.

The state vector itself is encompassed by the `Computer` class, which stores a complex vector `coeff_`, as well as a vector of `QubitBasis` objects (both of dimension $2^{n_\mathrm{qb}}$).

One can then apply a `Gate`, `Circuit`, or `QubitOperator` to transform the state vector by modifying `coeff_`.
We will demonstrate some key examples of this in this tutorial.


## A note on the QuantumBasis class



The `QubitBasis` class represents tensor product elements of the Fock space basis (in particle number representation).
Notably, a single basis is stored by the `state_` attribute, and is of type 64 bit unsigned long.
Using this data type allows for efficient bitwise operations.

Usage of `QubitBasis` is captured by the following example:

> Instantiate a `QubitBasis` object that gives the binary representation of several integers [0, 1, 2, and 12], and print the basis with respect to a six qubit system. Note that a `QubitBasis` must be printed using the `str(x)` function, where `x` is an intager specifying the number of qubits to print. 

In [2]:
qbasis_0 = qf.QubitBasis(0)
print(qbasis_0.str(6), '\n')

qbasis_1 = qf.QubitBasis(1)
print(qbasis_1.str(6), '\n')

qbasis_2 = qf.QubitBasis(2)
print(qbasis_2.str(6), '\n')

qbasis_12 = qf.QubitBasis(12)
print(qbasis_12.str(6), '\n')

|000000> 

|100000> 

|010000> 

|001100> 



> Flip the bit in position 3rd of qbasis_12.

In [3]:
qbasis_12.flip_bit(2)
print(qbasis_12.str(6), '\n')

|000100> 



## The QuantumComputer class

The `Computer` class is the central class in QForte in that it is used for essentially all applications. The core attributes of this class are the vector of complex coefficients `coeff_`, and the corresponding vector of `QubitBasis` objects that represent each basis element of the Fock space.  

> Instantiate a `Computer` with four qubits and print the representation. Note that we always initialize to the vacuum.

In [4]:
nqb = 4
qcomp = qf.Computer(nqb)

# qf.smart_print(qcomp)
print(qcomp)

Computer(
+1.000000 |0000>
)


## Manipulating the state vector

Once the `Computer` is initialized we can manipulate the state vector by applying `Gate`s, `Circuit`s and `QuantumOperaotrs`s. As a small example we will demonstrate how to construct the two qubit Bell state 
\begin{equation}
\ket{\Psi_\rm{Bell}} = \frac{1}{\sqrt{2}}\ket{00} + \frac{1}{\sqrt{2}}\ket{11}
\end{equation}
using QForte.

Recall that the action of the controlled $\hat{X}$ Pauli gate [with target qubit 0, and control qubit 1 ($c\hat{X}_{0,1}$)] is:
\begin{equation}
c\hat{X}_{0,1}\ket{00} = \ket{00}
\end{equation}
\begin{equation}
c\hat{X}_{0,1}\ket{01} = \ket{11},
\end{equation}
Recall that the action of the Hadamard gate $\hat{H}$ is:
\begin{equation}
\hat{H}\ket{0} = \frac{1}{\sqrt{2}} \big( \ket{0} + \ket{1} \big)
\end{equation}
\begin{equation}
\hat{H}\ket{1} = \frac{1}{\sqrt{2}} \big( \ket{0} - \ket{1} \big)
\end{equation}

> Initialize the bell state using elementary gates.

In [5]:
# First, initialize and print the state of the QuantumComputer.
nqb = 2
qbell = qf.Computer(nqb)
print(qbell)

# Initailize the gates needed to build the Bell state.
H_0 = qf.gate('H', 0)
cX_0_1 = qf.gate('cX', 1 , 0)

# Apply the Hadamrad gate and print.
qbell.apply_gate(H_0)
print(qbell)

# Finally, apply the CNOT (cX) gate and print the Bell state.
qbell.apply_gate(cX_0_1)
print(qbell)

Computer(
+1.000000 |00>
)
Computer(
+0.707107 |00>
+0.707107 |10>
)
Computer(
+0.707107 |00>
+0.707107 |11>
)


## Another circuit design exercise 

Another useful circuit $\hat{U}_\mathrm{split}$ is one which is able to prepare a simple superposition of two quantum basis states 
\begin{equation}
\ket{\Phi_I} = \ket{q_0q_1..q_{n-1}}; q_i \in \{0,1\} 
\end{equation}

such that the state on the quantum computer is given by

\begin{equation}
\ket{\Psi} = \frac{1}{\sqrt{2}} \ket{\Phi_I} + \frac{1}{\sqrt{2}} \ket{\Phi_J} = \hat{U}_\mathrm{split} \ket{\bar{0}}.
\end{equation}

Some examples are determination of (certain) off diagonal matrix elements, or preparation of references with more than one basis state in quantum simulation algorithms. 

> Write a function that takes in two bit lists A and B (representing the qubit configurations of $\Phi_I$ and $\Phi_J$ that are of equal length and particle-number) and returns the circuit $\hat{U}_\mathrm{split}$. Test this circuit and print the resulting state. **HINT** it may useful to base your strategy on the above construction of the Bell state, and you may also want to use the "open" CNOT gate
\begin{equation}
oc\hat{X}_{\mathrm{t},\mathrm{c}}  = \hat{X}_\mathrm{c} c \hat{X}_\mathrm{t,c} \hat{X}_\mathrm{c}
\end{equation}
which has the action, 
\begin{equation}
oc\hat{X}_{0,1}\ket{00} = \ket{10}
\end{equation}
\begin{equation}
oc\hat{X}_{0,1}\ket{01} = \ket{01}.
\end{equation}

In [6]:
def get_Usplit(A, B):
    
    # Define the return circuit
    Usplit = qf.Circuit()
    
    if (len(A) != len(B)) or (sum(A) != sum(B)) or (A==B):
        raise ValueError("A and B must have the same length, the same particle-number, and be different states.")
        
    nqb = len(A)
        
    # Make list of dissimilar bits and which state has dissimilar bit set to 1.
    diff_bits = []
    ones_bits = []
    for i in range(nqb):
        if(A[i] != B[i]):
            if(A[i]):
                diff_bits.append((i,'A'))
            else:
                diff_bits.append((i,'B'))
        
        elif(A[i]==B[i]==1):       
            ones_bits.append(i)    
    
    # Add the Hadamard gate that will split the state around the first dissimilar qubit
    Usplit.add_gate(qf.gate('H', diff_bits[0][0]))
    
    for k in range(1, len(diff_bits)):
        
        if diff_bits[k][1]==diff_bits[k-1][1]:
            Usplit.add_gate(qf.gate('cX',diff_bits[k][0], diff_bits[k-1][0]))
        
        else:
            Usplit.add_gate(qf.gate('X',diff_bits[k-1][0]))
            Usplit.add_gate(qf.gate('cX',diff_bits[k][0],diff_bits[k-1][0]))
            Usplit.add_gate(qf.gate('X',diff_bits[k-1][0]))
    
    # Finally flip all the bits that are supposed to be 1.
    for p in ones_bits:
        Usplit.add_gate(qf.gate('X', p))
    
    return Usplit
    
    

Now test your funciton

In [7]:
# Try these
A1 = [1,1,0,0]
B1 = [0,0,1,1]

A2 = [1,1,0,0,1,1,0,1]
B2 = [1,1,1,1,0,1,0,0]

Usplit = get_Usplit(A1, B1) 
qc = qf.Computer(len(A1))
qc.apply_circuit(Usplit)
print(qc)

Computer(
+0.707107 |1100>
+0.707107 |0011>
)
